In [10]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import grid_search
from sklearn.linear_model import LogisticRegression

In [33]:
traindf = pd.read_json("train.json")
traindf['ingredients_clean_string'] = [' , '.join(z).strip() for z in traindf['ingredients']]  
traindf['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in traindf['ingredients']]       

testdf = pd.read_json("test.json") 
testdf['ingredients_clean_string'] = [' , '.join(z).strip() for z in testdf['ingredients']]
testdf['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in testdf['ingredients']]       

y= traindf['cuisine']
X = traindf

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [34]:
X.head()

,cuisine,id,ingredients,ingredients_clean_string,ingredients_string
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...","romaine lettuce , black olives , grape tomatoe...",romaine lettuce black olives grape tomatoes ga...
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...","plain flour , ground pepper , salt , tomatoes ...",plain flour ground pepper salt tomato ground b...
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...","eggs , pepper , salt , mayonaise , cooking oil...",egg pepper salt mayonaise cooking oil green ch...
3,indian,22213,"[water, vegetable oil, wheat, salt]","water , vegetable oil , wheat , salt",water vegetable oil wheat salt
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...","black pepper , shallots , cornflour , cayenne ...",black pepper shallot cornflour cayenne pepper ...


In [35]:
traindf = X_train
testdf = X_test
testdf.head()

,cuisine,id,ingredients,ingredients_clean_string,ingredients_string
21513,chinese,7958,"[pork, cooking oil, bamboo shoots, chinese ric...","pork , cooking oil , bamboo shoots , chinese r...",pork cooking oil bamboo shoots chinese rice wi...
1796,spanish,36179,"[hog casings, hungarian paprika, ancho powder,...","hog casings , hungarian paprika , ancho powder...",hog casings hungarian paprika ancho powder rio...
21861,greek,8331,"[lamb stock, lemon, lamb shoulder, onions, gro...","lamb stock , lemon , lamb shoulder , onions , ...",lamb stock lemon lamb shoulder onion ground ci...
26571,indian,41097,"[green peas, cinnamon sticks, clove, chopped o...","green peas , cinnamon sticks , clove , chopped...",green peas cinnamon sticks clove chopped onion...
28720,italian,2007,"[vegetable oil spray, cumin seed, grated parme...","vegetable oil spray , cumin seed , grated parm...",vegetable oil spray cumin seed grated parmesan...


In [36]:
corpustr = traindf['ingredients_string']
vectorizertr = TfidfVectorizer(stop_words='english',
                             ngram_range = ( 1 , 1 ),analyzer="word", 
                             max_df = .57 , binary=False , token_pattern=r'\w+' , sublinear_tf=False)

tfidftr=vectorizertr.fit_transform(corpustr).todense()

corpusts = testdf['ingredients_string']
vectorizerts = TfidfVectorizer(stop_words='english')

tfidfts=vectorizertr.transform(corpusts)

predictors_tr = tfidftr

targets_tr = traindf['cuisine']

predictors_ts = tfidfts


In [40]:

#classifier = LinearSVC(C=0.80, penalty="l2", dual=False)
parameters = {'C':[1, 10]}
#clf = LinearSVC()

clf = LogisticRegression()

classifier = grid_search.GridSearchCV(clf, parameters)

classifier=classifier.fit(predictors_tr,targets_tr)

predictions=classifier.predict(predictors_ts)

testdf['cuisine'] = predictions
testdf = testdf.sort_index()

testdf[['id' , 'ingredients_clean_string' , 'cuisine' ]].to_csv("submission.csv")

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.7904462602137021